In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
torch.manual_seed(42)

In [ ]:
N = 256
l = 8
d = 8
block_nums = N // l #32
dim = 64
heads = 4
head_dim = dim//heads
batch_size = 1

In [ ]:
X = torch.randn(batch_size, N, dim)
Wq = torch.randn(dim, dim)
Wk = torch.randn(dim, dim)
Wv = torch.randn(dim, dim)

Q = X @ Wq
K = X @ Wk
V = X @ Wv

In [ ]:
print(Q.shape)
print(K.shape)
print(V.shape)

torch.Size([1, 256, 64])
torch.Size([1, 256, 64])
torch.Size([1, 256, 64])


In [ ]:
d = 4
max_idx = round((N - l) / d) # block max idx
print(max_idx)
print(torch.arange(max_idx) * d + 1)
print(torch.arange(max_idx) * d + l)

62
tensor([  1,   5,   9,  13,  17,  21,  25,  29,  33,  37,  41,  45,  49,  53,
         57,  61,  65,  69,  73,  77,  81,  85,  89,  93,  97, 101, 105, 109,
        113, 117, 121, 125, 129, 133, 137, 141, 145, 149, 153, 157, 161, 165,
        169, 173, 177, 181, 185, 189, 193, 197, 201, 205, 209, 213, 217, 221,
        225, 229, 233, 237, 241, 245])
tensor([  8,  12,  16,  20,  24,  28,  32,  36,  40,  44,  48,  52,  56,  60,
         64,  68,  72,  76,  80,  84,  88,  92,  96, 100, 104, 108, 112, 116,
        120, 124, 128, 132, 136, 140, 144, 148, 152, 156, 160, 164, 168, 172,
        176, 180, 184, 188, 192, 196, 200, 204, 208, 212, 216, 220, 224, 228,
        232, 236, 240, 244, 248, 252])


In [ ]:
d = l
max_idx = round((N) / d) # block max idx
print(max_idx)
print(torch.arange(max_idx) * d + 1)
print(torch.arange(max_idx) * d + l)

32
tensor([  1,   9,  17,  25,  33,  41,  49,  57,  65,  73,  81,  89,  97, 105,
        113, 121, 129, 137, 145, 153, 161, 169, 177, 185, 193, 201, 209, 217,
        225, 233, 241, 249])
tensor([  8,  16,  24,  32,  40,  48,  56,  64,  72,  80,  88,  96, 104, 112,
        120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224,
        232, 240, 248, 256])


In [ ]:
W_pe = torch.randn(l, dim)
phi_K = torch.randn(l, 1)
phi_V = torch.randn(l, 1)

In [ ]:
K_cmp = []
V_cmp = []
for i in range(max_idx):
  cur_K = K[:, i*d: i*d+l, :] + W_pe.unsqueeze(0)
  cur_V = V[:, i*d: i*d+l, :] + W_pe.unsqueeze(0)
  cur_K = cur_K.transpose(1, 2) @ phi_K
  cur_V = cur_V.transpose(1, 2) @ phi_V
  K_cmp.append(cur_K)
  V_cmp.append(cur_V)



In [ ]:
K_cmp[0].shape

torch.Size([1, 64, 1])

In [ ]:
K_cmp = torch.cat(K_cmp, dim=2).transpose(1,2)
V_cmp = torch.cat(V_cmp, dim=2).transpose(1,2)
print(K_cmp.shape)

torch.Size([1, 32, 64])


In [ ]:
Q_mha = Q.view(1, N, heads, head_dim).transpose(1,2)
K_mha = K_cmp.view(1, block_nums, heads, head_dim).transpose(1,2)
V_mha = V_cmp.view(1, block_nums, heads, head_dim).transpose(1,2)
score_cmp = Q_mha @ K_mha.transpose(2,3)

In [ ]:
score_cmp.shape

torch.Size([1, 4, 256, 32])

In [ ]:
p_cmp = F.softmax(score_cmp, dim=-1)
print(p_cmp.shape)
o_cmp = p_cmp @ V_mha
print(o_cmp.shape)
o_cmp = o_cmp.transpose(2, 1).reshape(batch_size, N, dim)
print(o_cmp.shape)

torch.Size([1, 4, 256, 32])
torch.Size([1, 4, 256, 16])
torch.Size([1, 256, 64])


## Selector

In [ ]:
p_slc = p_cmp.sum(dim=1)
print(p_slc.shape)
select_top_k = 3
value, idx = torch.topk(p_slc, dim=2, k=select_top_k)
print(value.shape)
print(idx.shape)

torch.Size([1, 256, 32])
torch.Size([1, 256, 3])
torch.Size([1, 256, 3])


In [ ]:
K_slc = torch.zeros(batch_size, N, select_top_k * d, dim)
V_slc = torch.zeros(batch_size, N, select_top_k * d, dim)
idx_slc_start = idx * d
idx_slc_end = idx * d + l

for i in range(batch_size):
  for j in range(N):
    for k in range(select_top_k):
      K_slc[i, j, k*d: k*d+l, :] = K[i, idx_slc_start[i, j, k] : idx_slc_end[i, j, k], :]
      V_slc[i, j, k*d: k*d+l, :] = V[i, idx_slc_start[i, j, k] : idx_slc_end[i, j, k], :]
print(K_slc.shape)
print(V_slc.shape)

torch.Size([1, 256, 24, 64])
torch.Size([1, 256, 24, 64])


In [ ]:
K_mha = K_slc.view(batch_size, N, heads, select_top_k * d, head_dim).transpose(1, 2)
V_mha = V_slc.view(batch_size, N, heads, select_top_k * d, head_dim).transpose(1, 2)
print(Q_mha.shape) #bs, nh, N, h_dim
print(K_mha.shape) # bs, nh, N, top_k * d, h_dim
print(V_mha.shape) # bs, nh, N, top_k * d, h_dim

torch.Size([1, 4, 256, 16])
torch.Size([1, 4, 256, 24, 16])
torch.Size([1, 4, 256, 24, 16])


In [ ]:
score_slc = (Q_mha.unsqueeze(-2) @ K_mha.transpose(-2,-1))
print(score_slc.shape)
p_slc = F.softmax(score_slc, dim=-1)
o_slc = (p_slc @ V_mha).squeeze(-2)
print(o_slc.shape)
o_slc = o_slc.transpose(1, 2).reshape(batch_size, N, dim)
print(o_slc.shape)

torch.Size([1, 4, 256, 1, 24])
torch.Size([1, 4, 256, 16])
torch.Size([1, 256, 64])


# Sliding Window

In [ ]:
def get_window_mask(seq_len, window_len):
  mask = torch.ones(seq_len, seq_len)
  mask = torch.tril(mask)
  wdw_mask = torch.ones(seq_len-window_len, seq_len-window_len)
  wdw_mask = 1.0 - torch.tril(wdw_mask)
  mask[window_len:, :seq_len-window_len] = wdw_mask
  return mask

In [ ]:
window_size = 16
score_wdw = Q @ K.transpose(-2, -1) / math.sqrt(dim)
score_wdw = score_wdw * get_window_mask(N, window_size)
p_wdw = F.softmax(score_wdw, dim=-1)
o_wdw = p_wdw @ V
o_wdw.shape

torch.Size([1, 256, 64])

In [ ]:
W_gate = torch.randn(dim, 3)
gate = X @ W_gate
gate = F.sigmoid(gate)
gate = gate.unsqueeze(-2)
print(gate.shape)

torch.Size([1, 256, 1, 3])


In [ ]:
o_list = torch.stack([o_cmp, o_slc, o_wdw], dim=-1)
print(o_list.shape)
a = (o_list * gate).sum(dim=-1)
print(a.shape)

torch.Size([1, 256, 64, 3])
torch.Size([1, 256, 64])


In [ ]:
# [1, 256, 64]